# Student's t-test for paired samples in Julia

In [1]:
using BenchmarkTools

# Import Julia functions from lib
include("../lib_stat/lib_stat.jl")
using .lib_stat: avevar, betai

# Compile fortran source code (using a Python script)
run(`python ../lib_stat/compile_fortran.py`)

Process(`python ../lib_stat/compile_fortran.py`, ProcessExited(0))

Data input:

In [2]:
data1 = [70.0, 71.0, 72.0, 73.0, 70.5]
data2 = [70.15, 71.05, 71.95, 72.85, 70.45];

Function definition:

In [3]:
function tptest(data1, data2)
    # Given the arrays "data1" and "data2" of length "n", returns
    # the Student's "t" for paired data and its significance as "prob".

    # Small values of "prob" indicates that the arrays have different
    # means.

    n1 = length(data1)
    n2 = length(data2)
  
    if (n1 != n2)
        print("Data arrays must have the same size")
    else
        n = n1
    end

    ave1, var1 = avevar(data1)
    ave2, var2 = avevar(data2)

    cov = 0.0
    for i = 1:n
      cov = cov + (data1[i] - ave1) * (data2[i] - ave2)
    end

    df = n - 1.0
    cov = cov / df

    # Student's t
    t = (ave1 - ave2) / sqrt((var1 + var2 - 2.0 * cov) / n)

    # significance
    prob = betai(0.5 * df, 0.5, df/(df + t^2))
    return t, prob
end;

#### Calling the Julia tutest function:

In [4]:
t, prob = tptest(data1, data2)

print(t, " ",prob)

0.1961161351382784 0.8540797033791304

#### Calling the Fortran tutest function:

In [5]:
run(`../lib_stat/fmain_tptest`);

  0.19611613513827841       0.85407970337913042     


# Timing against Fortran:

Test function in **Julia** and timing:

In [6]:
function speed_test()
    for i = 1:1e3
        tptest(data1, data2)
        end
    end;

@btime speed_test()

  383.930 μs (4000 allocations: 406.25 KiB)


Timing in **Fortran**:

In [7]:
@btime run(`../lib_stat/fspeed_tptest`);

  1.553 ms (46 allocations: 1.84 KiB)
